In [1]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Define a range for green color
            lower_green = np.array([35, 50, 50])
            upper_green = np.array([85, 255, 255])

            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw bounding boxes around detected green regions
            for contour in contours:
                if cv2.contourArea(contour) > 500:  # Ignore small contours to reduce noise
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

# Paths to the directories
input_directory = "D:\Projects\Weed Detection\input"
output_directory = "D:\Projects\Weed Detection\output"
detect_green_areas(input_directory, output_directory)


In [2]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Adjusted broader range for green color
            lower_green = np.array([20, 40, 40])
            upper_green = np.array([95, 255, 255])

            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Morphological operations to remove noise
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=2)

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Draw bounding boxes around detected green regions
            for contour in contours:
                if cv2.contourArea(contour) > 300:  # Reduced minimum area
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

# Paths to the directories
input_directory = "D:\Projects\Weed Detection\input"
output_directory = "D:\Projects\Weed Detection\output"
detect_green_areas(input_directory, output_directory)


In [9]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Further expanded range for green color
            lower_green = np.array([73, 15, 15])
            upper_green = np.array([100, 255, 255])

            # Create a binary mask where green regions are white
            mask = cv2.inRange(hsv, lower_green, upper_green)

            # Morphological operations to remove noise
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=2)

            # Find contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Get bounding boxes from contours
            rects = [cv2.boundingRect(contour) for contour in contours if cv2.contourArea(contour) > 100]

            # Merge intersecting boxes
            merged_rects, _ = cv2.groupRectangles(rects + rects, groupThreshold=1, eps=0.5)

            # Draw merged bounding boxes
            for x, y, w, h in merged_rects:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

# Paths to the directories
input_directory = "D:\Projects\Weed Detection\input"
output_directory = "D:\Projects\Weed Detection\output"
detect_green_areas(input_directory, output_directory)


In [14]:
import cv2
import numpy as np
import os

def merge_intersecting_rectangles(rects):
    merged = True
    while merged:
        merged = False
        new_rects = []
        used = set()
        
        for i, (x1, y1, w1, h1) in enumerate(rects):
            if i in used:
                continue
            merged_this_round = False
            for j, (x2, y2, w2, h2) in enumerate(rects):
                if i != j and j not in used:
                    # Check if the two rectangles intersect
                    if x1 < x2 + w2 and x1 + w1 > x2 and y1 < y2 + h2 and y1 + h1 > y2:
                        new_rects.append((min(x1, x2), min(y1, y2), max(x1 + w1, x2 + w2) - min(x1, x2), max(y1 + h1, y2 + h2) - min(y1, y2)))
                        merged = True
                        merged_this_round = True
                        used.add(i)
                        used.add(j)
                        break
            
            if not merged_this_round:
                new_rects.append((x1, y1, w1, h1))
                used.add(i)
        
        rects = new_rects
    
    return rects

def detect_green_areas(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_dir, filename)
            
            image = cv2.imread(image_path)
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            lower_green = np.array([20, 25, 25])
            upper_green = np.array([100, 255, 255])
            mask = cv2.inRange(hsv, lower_green, upper_green)
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=2)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rects = [cv2.boundingRect(contour) for contour in contours if cv2.contourArea(contour) > 100]
            merged_rects = merge_intersecting_rectangles(rects)
            for x, y, w, h in merged_rects:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

# Paths to the directories
input_directory = "D:\Projects\Weed Detection\input"
output_directory = "D:\Projects\Weed Detection\output"
detect_green_areas(input_directory, output_directory)


In [36]:
import cv2
import numpy as np
import os

def merge_intersecting_rectangles(rects):
    merged = True
    while merged:
        merged = False
        new_rects = []
        used = set()
        
        for i, (x1, y1, w1, h1) in enumerate(rects):
            if i in used:
                continue
            merged_this_round = False
            for j, (x2, y2, w2, h2) in enumerate(rects):
                if i != j and j not in used:
                    # Check if the two rectangles intersect
                    if x1 < x2 + w2 and x1 + w1 > x2 and y1 < y2 + h2 and y1 + h1 > y2:
                        new_rects.append((min(x1, x2), min(y1, y2), max(x1 + w1, x2 + w2) - min(x1, x2), max(y1 + h1, y2 + h2) - min(y1, y2)))
                        merged = True
                        merged_this_round = True
                        used.add(i)
                        used.add(j)
                        break
            
            if not merged_this_round:
                new_rects.append((x1, y1, w1, h1))
                used.add(i)
        
        rects = new_rects
    
    return rects

def increase_saturation(image, boost_factor=1.3):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * boost_factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


def detect_green_areas(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(input_dir, filename)
            
            image = cv2.imread(image_path)
            image = increase_saturation(image)
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            lower_green = np.array([30, 30, 30])
            upper_green = np.array([100, 255, 255])
            mask = cv2.inRange(hsv, lower_green, upper_green)
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.erode(mask, kernel, iterations=1)
            mask = cv2.dilate(mask, kernel, iterations=2)
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            rects = [cv2.boundingRect(contour) for contour in contours if cv2.contourArea(contour) > 100]
            merged_rects = merge_intersecting_rectangles(rects)
            for x, y, w, h in merged_rects:
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

# Paths to the directories
input_directory = "D:\Projects\Weed Detection\input"
output_directory = "D:\Projects\Weed Detection\output"
detect_green_areas(input_directory, output_directory)

In [4]:
import cv2
import numpy as np
import os

def merge_intersecting_rectangles(rects):
    merged = True
    while merged:
        merged = False
        new_rects = []
        used = set()
        
        for i, (x1, y1, w1, h1) in enumerate(rects):
            if i in used:
                continue
            merged_this_round = False
            for j, (x2, y2, w2, h2) in enumerate(rects):
                if i != j and j not in used:
                    # Check if the two rectangles intersect
                    if x1 < x2 + w2 and x1 + w1 > x2 and y1 < y2 + h2 and y1 + h1 > y2:
                        new_rects.append((min(x1, x2), min(y1, y2), max(x1 + w1, x2 + w2) - min(x1, x2), max(y1 + h1, y2 + h2) - min(y1, y2)))
                        merged = True
                        merged_this_round = True
                        used.add(i)
                        used.add(j)
                        break
            
            if not merged_this_round:
                new_rects.append((x1, y1, w1, h1))
                used.add(i)
        
        rects = new_rects
    
    return rects

def increase_saturation(image, boost_factor=1.3):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * boost_factor, 0, 255)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def process_frame(frame):
    # Increase saturation
    frame = increase_saturation(frame)
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_green = np.array([30, 30, 30])
    upper_green = np.array([100, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    mask = cv2.dilate(mask, kernel, iterations=2)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rects = [cv2.boundingRect(contour) for contour in contours if cv2.contourArea(contour) > 100]
    merged_rects = merge_intersecting_rectangles(rects)
    
    for x, y, w, h in merged_rects:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return frame

def detect_green_in_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (int(cap.get(3)), int(cap.get(4))))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        processed_frame = process_frame(frame)
        out.write(processed_frame)

    cap.release()
    out.release()

# Paths to the videos
input_video_path = r"D:\Projects\Weed Detection\video input\20231005_165420.mp4"
output_video_path = r"D:\Projects\Weed Detection\video input\output_video.mp4"
detect_green_in_video(input_video_path, output_video_path)

In [40]:
cap = cv2.VideoCapture(input_video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame(frame)
    out.write(processed_frame)

cap.release()
out.release()


In [2]:
cap = cv2.VideoCapture(input_video_path)

if not cap.isOpened():
    print("Error: Couldn't open the input video.")


fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using 'mp4v' codec
out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

if not out.isOpened():
    print("Error: Couldn't open the output video for writing. Check your output path or codec.")


frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    processed_frame = process_frame(frame)
    out.write(processed_frame)

    frame_count += 1
    if frame_count % 50 == 0:  # Print a message every 50 frames
        print(f"Processed {frame_count} frames.")

cap.release()
out.release()
print(f"Processing completed. {frame_count} frames processed.")



Error: Couldn't open the input video.
Error: Couldn't open the output video for writing. Check your output path or codec.
Processing completed. 0 frames processed.


In [1]:
import cv2

# Open the video file
video_path = "D:\Projects\Weed Detection\video input\20231005_165420.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Couldn't open the video.")
    exit()

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    # Break the loop if we've reached the end of the video
    if not ret:
        break

    # Display the frame in a window called "Video"
    cv2.imshow('Video', frame)

    # Wait for 30ms or until the user presses the 'q' key to move to the next frame
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error: Couldn't open the video.


In [1]:
import tensorflow as tf

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [10]:
pip install tensorflow==2.10.0

     ---------------------------------------- 0.0/48.9 kB ? eta -:--:--
     ------------------------- -------------- 30.7/48.9 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 48.9/48.9 kB 822.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/455.9 MB 1.3 MB/s eta 0:05:57
   ---------------------------------------- 0.1/455.9 MB 1.3 MB/s eta 0:05:51
   ---------------------------------------- 0.2/455.9 MB 1.3 MB/s eta 0:05:50
   ---------------------------------------- 0.2/455.9 MB 1.2 MB/s eta 0:06:06
   ---------------------------------------- 0.3/455.9 MB 1.3 MB/s eta 0:06:02
   ---------------------------------------- 0.3/455.9 MB 1.3 MB/s eta 0:05:48
   ---------------------------------------- 0.4/455.9 MB 1.2 MB/s eta 0:06:07
   ---------------------------------------- 0.5/455.9 MB 1.3 MB/s eta 0:05:56
   ---------------------------------------- 0.5/455.9 MB 1.3 MB/s eta 0:0

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.13.0 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.10.0 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.


In [4]:
pip install protobuf==3.20

   ---------------------------------------- 0.0/904.1 kB ? eta -:--:--
   - -------------------------------------- 30.7/904.1 kB 1.3 MB/s eta 0:00:01
   ---- ----------------------------------- 92.2/904.1 kB 1.1 MB/s eta 0:00:01
   ------- -------------------------------- 163.8/904.1 kB 1.2 MB/s eta 0:00:01
   --------- ------------------------------ 204.8/904.1 kB 1.2 MB/s eta 0:00:01
   ------------ --------------------------- 276.5/904.1 kB 1.2 MB/s eta 0:00:01
   -------------- ------------------------- 337.9/904.1 kB 1.2 MB/s eta 0:00:01
   ----------------- ---------------------- 389.1/904.1 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 450.6/904.1 kB 1.2 MB/s eta 0:00:01
   ----------------------- ---------------- 522.2/904.1 kB 1.3 MB/s eta 0:00:01
   ------------------------- -------------- 583.7/904.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------- ----------- 655.4/904.1 kB 1.3 MB/s eta 0:00:01
   ------------------------------ --------- 696.3/9

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.10.0 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.0 which is incompatible.
tensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.10.1 which is incompatible.
tensorflow-intel 2.13.0 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-estimator 2.10.0 which is incompatible.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.9.0 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorflow 2.10.0 requires protobuf<3.20,>=